# MOAI-LLM 기본 학습 가이드

이 노트북은 MOAI-LLM의 전체 학습 파이프라인을 단계별로 실행합니다:

1. **환경 확인** - GPU, 라이브러리 설정
2. **토크나이저 학습** - HuggingFace 데이터셋 기반
3. **사전학습 (Pretrain)** - 언어 모델 기초 학습
4. **파인튜닝 (SFT)** - Instruction following 학습
5. **추론 테스트** - 학습된 모델 사용

**권장 환경**: Google Colab (T4/V100), Kaggle Notebook, 또는 로컬 GPU

## 0. 환경 설정 및 확인

In [ ]:
# GPU 및 환경 확인
import torch
import sys
from pathlib import Path

print("=" * 60)
print("환경 정보")
print("=" * 60)
print(f"Python 버전: {sys.version.split()[0]}")
print(f"PyTorch 버전: {torch.__version__}")
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"CUDA 버전: {torch.version.cuda}")
    print(f"GPU 개수: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"  GPU {i}: {torch.cuda.get_device_name(i)}")
        mem_total = torch.cuda.get_device_properties(i).total_memory / 1e9
        print(f"    메모리: {mem_total:.1f} GB")
else:
    print("⚠️ GPU를 사용할 수 없습니다. CPU로 실행됩니다 (매우 느림).")

print("=" * 60)

In [ ]:
# MOAI-LLM 설치 (로컬에서 실행 시)
# Colab/Kaggle에서는 먼저 저장소를 클론하세요

# Google Colab/Kaggle인 경우 주석 해제:
# !git clone https://github.com/sh2orc/moai-llm.git
# %cd moai-llm
# !pip install -e .
# !pip install sentencepiece datasets accelerate

# Flash Attention 설치 (선택적, 속도 향상)
# !pip install flash-attn --no-build-isolation

## 1. 토크나이저 학습 (10분)

HuggingFace Wikipedia 데이터셋으로 토크나이저를 학습합니다.

**빠른 테스트**: 10,000 샘플, vocab_size=32K  
**실전 학습**: 전체 데이터, vocab_size=128K

In [ ]:
%%bash

# 빠른 테스트 (2-3분, 추천)
python train_tokenizer.py \
    --dataset wikipedia \
    --dataset_config 20220301.ko \
    --vocab_size 32000 \
    --max_samples 10000 \
    --output_dir tokenizers/test-32k

# 실전 학습 (1-2시간, 주석 해제)
# python train_tokenizer.py \
#     --dataset wikipedia \
#     --dataset_config 20220301.ko \
#     --vocab_size 128000 \
#     --output_dir tokenizers/ko-128k

In [ ]:
%%bash

# BCCard 데이터셋으로 금융 도메인 특화 토크나이저 학습
python train_tokenizer.py \
    --dataset BCCard/BCCard-Finance-Kor-QnA \
    --vocab_size 32000 \
    --max_samples 10000 \
    --output_dir tokenizers/finance-32k

# 또는 Wikipedia + BCCard 혼합 학습 (더 균형잡힌 토크나이저)
# python train_tokenizer.py \
#     --dataset wikipedia BCCard/BCCard-Finance-Kor-QnA \
#     --dataset_config 20220301.ko None \
#     --vocab_size 32000 \
#     --max_samples 5000 \
#     --output_dir tokenizers/mixed-32k

### 1.1. BCCard 데이터셋으로 토크나이저 학습

금융 도메인에 특화된 토크나이저를 만들기 위해 BCCard 데이터셋을 사용할 수도 있습니다.

In [ ]:
# 토크나이저 테스트
import sentencepiece as spm

# 학습된 토크나이저 로드
tokenizer_path = "tokenizers/test-32k/tokenizer.model"
sp = spm.SentencePieceProcessor()
sp.load(tokenizer_path)

# 테스트 문장
test_texts = [
    "안녕하세요. MOAI-LLM 토크나이저 테스트입니다.",
    "Hello, this is a tokenizer test.",
    "機械学習とは何ですか？",  # 일본어
]

print("=" * 60)
print("토크나이저 테스트")
print("=" * 60)
print(f"어휘 크기: {sp.vocab_size():,}")
print(f"BOS 토큰 ID: {sp.bos_id()}")
print(f"EOS 토큰 ID: {sp.eos_id()}")
print(f"PAD 토큰 ID: {sp.pad_id()}")
print()

for text in test_texts:
    tokens = sp.encode(text)
    pieces = sp.encode_as_pieces(text)
    decoded = sp.decode(tokens)
    
    print(f"원문: {text}")
    print(f"토큰 개수: {len(tokens)}")
    print(f"토큰 조각: {pieces[:10]}..." if len(pieces) > 10 else f"토큰 조각: {pieces}")
    print(f"복원: {decoded}")
    print("-" * 60)

In [ ]:
%%bash

# 방법 1: 체크포인트에서 재개 (같은 output_dir 사용)
python train.py \
    --mode pretrain \
    --dataset wikitext \
    --dataset_config wikitext-2-raw-v1 \
    --tokenizer_path tokenizers/test-32k \
    --output_dir outputs/pretrain-test \
    --resume_from_checkpoint outputs/pretrain-test/checkpoint-50 \
    --max_steps 200 \
    --batch_size 2 \
    --logging_steps 10 \
    --save_steps 50

# 방법 2: 기존 모델을 로드하여 추가 학습 (새로운 output_dir 사용)
# python train.py \
#     --mode pretrain \
#     --dataset wikipedia \
#     --dataset_config 20220301.ko \
#     --tokenizer_path tokenizers/test-32k \
#     --pretrained_model outputs/pretrain-test/final_model \
#     --output_dir outputs/pretrain-continued \
#     --max_steps 500 \
#     --batch_size 2 \
#     --logging_steps 10 \
#     --save_steps 100

### 2.1. 기존 모델에서 이어서 학습하기 (Resume Training)

학습이 중단되었거나 추가 학습이 필요한 경우 체크포인트에서 재개할 수 있습니다.

## 2. 사전학습 (Pretrain)

언어 모델의 기초를 학습합니다.

**빠른 테스트**: 100 steps (5분)  
**실전 학습**: 수천~수만 steps (수일)

In [ ]:
%%bash

# 방법 1: 체크포인트에서 재개
python train.py \
    --mode sft \
    --dataset BCCard/BCCard-Finance-Kor-QnA \
    --tokenizer_path tokenizers/test-32k \
    --pretrained_model outputs/pretrain-test/final_model \
    --output_dir outputs/sft-test \
    --resume_from_checkpoint outputs/sft-test/checkpoint-25 \
    --max_steps 100 \
    --batch_size 2 \
    --logging_steps 10 \
    --save_steps 25

# 방법 2: 기존 SFT 모델에 다른 데이터셋으로 추가 파인튜닝
# python train.py \
#     --mode sft \
#     --dataset your/another-qa-dataset \
#     --tokenizer_path tokenizers/test-32k \
#     --pretrained_model outputs/sft-test/final_model \
#     --output_dir outputs/sft-continued \
#     --num_epochs 1 \
#     --batch_size 2 \
#     --logging_steps 10 \
#     --save_steps 50

# 방법 3: Learning rate를 낮춰서 fine-tuning (catastrophic forgetting 방지)
# python train.py \
#     --mode sft \
#     --dataset BCCard/BCCard-Finance-Kor-QnA \
#     --tokenizer_path tokenizers/test-32k \
#     --pretrained_model outputs/sft-test/final_model \
#     --output_dir outputs/sft-low-lr \
#     --learning_rate 1e-6 \
#     --num_epochs 1 \
#     --batch_size 2

### 3.2. SFT 학습 재개하기

SFT 학습도 체크포인트에서 재개하거나 기존 SFT 모델을 추가로 파인튜닝할 수 있습니다.

In [ ]:
%%bash

# 빠른 테스트 (100 steps, 5분)
python train.py \
    --mode pretrain \
    --dataset wikitext \
    --dataset_config wikitext-2-raw-v1 \
    --tokenizer_path tokenizers/test-32k \
    --output_dir outputs/pretrain-test \
    --max_steps 100 \
    --batch_size 2 \
    --logging_steps 10 \
    --save_steps 50

# 실전 학습 (주석 해제, GPU 메모리에 맞게 조정)
# python train.py \
#     --mode pretrain \
#     --dataset wikipedia \
#     --dataset_config 20220301.ko \
#     --tokenizer_path tokenizers/ko-128k \
#     --output_dir outputs/pretrain-ko \
#     --batch_size 4 \
#     --gradient_accumulation_steps 8 \
#     --bf16 \
#     --gradient_checkpointing \
#     --logging_steps 100 \
#     --save_steps 1000

In [ ]:
# 학습 로그 확인
import json
from pathlib import Path

log_file = Path("outputs/pretrain-test/training_log.jsonl")

if log_file.exists():
    print("=" * 60)
    print("학습 진행 상황")
    print("=" * 60)
    
    with open(log_file) as f:
        logs = [json.loads(line) for line in f]
    
    # 최근 10개 로그 표시
    for log in logs[-10:]:
        step = log.get('step', 'N/A')
        loss = log.get('loss', 'N/A')
        lr = log.get('learning_rate', 'N/A')
        print(f"Step {step}: Loss={loss:.4f}, LR={lr:.2e}")
else:
    print("⚠️ 학습 로그 파일을 찾을 수 없습니다.")

## 3. 파인튜닝 (SFT - Supervised Fine-Tuning)

사전학습된 모델을 Q&A 형식으로 파인튜닝합니다.

**빠른 테스트**: 50 steps (3분)  
**실전 학습**: 3 epochs (수시간)

In [ ]:
# 데이터 길이 분포 확인
import numpy as np

# 질문과 답변 길이 계산
sample_size = min(1000, len(dataset['train']))
samples = dataset['train'].select(range(sample_size))

# 필드명이 다를 수 있으므로 확인
first_sample = dataset['train'][0]
text_fields = [k for k, v in first_sample.items() if isinstance(v, str)]

print("=" * 60)
print(f"텍스트 길이 통계 (처음 {sample_size}개 샘플)")
print("=" * 60)

for field in text_fields:
    lengths = [len(sample[field]) for sample in samples]
    
    print(f"\n[{field}]")
    print(f"  평균 길이: {np.mean(lengths):.1f} 글자")
    print(f"  최소 길이: {np.min(lengths)} 글자")
    print(f"  최대 길이: {np.max(lengths)} 글자")
    print(f"  중앙값: {np.median(lengths):.1f} 글자")

In [ ]:
# 데이터 예시 확인
import random

print("=" * 60)
print("BCCard 데이터셋 샘플 (랜덤 5개)")
print("=" * 60)

# 랜덤하게 5개 샘플 선택
num_samples = min(5, len(dataset['train']))
random_indices = random.sample(range(len(dataset['train'])), num_samples)

for i, idx in enumerate(random_indices, 1):
    sample = dataset['train'][idx]
    
    print(f"\n[샘플 {i}]")
    
    # 데이터 구조에 따라 출력 (일반적으로 'instruction', 'input', 'output' 또는 'question', 'answer')
    for key, value in sample.items():
        if isinstance(value, str) and len(value) > 200:
            print(f"{key}: {value[:200]}...")
        else:
            print(f"{key}: {value}")
    
    print("-" * 60)

In [ ]:
# BCCard 데이터셋 로드
from datasets import load_dataset

print("BCCard 금융 Q&A 데이터셋 로딩 중...")
dataset = load_dataset("BCCard/BCCard-Finance-Kor-QnA")

print("=" * 60)
print("데이터셋 정보")
print("=" * 60)
print(f"데이터셋 구조: {dataset}")
print(f"\n학습 데이터 크기: {len(dataset['train']):,} 샘플")

# 데이터 필드 확인
if len(dataset['train']) > 0:
    print(f"\n데이터 필드: {dataset['train'].column_names}")
    print(f"첫 번째 샘플 키: {list(dataset['train'][0].keys())}")

### 3.1. BCCard 금융 Q&A 데이터셋 확인

SFT 학습에 사용할 [BCCard/BCCard-Finance-Kor-QnA](https://huggingface.co/datasets/BCCard/BCCard-Finance-Kor-QnA) 데이터셋을 먼저 살펴봅니다.

이 데이터셋은 금융 관련 한국어 질문-답변 쌍으로 구성되어 있습니다.

In [ ]:
%%bash

# 빠른 테스트 (50 steps, 3분)
python train.py \
    --mode sft \
    --dataset BCCard/BCCard-Finance-Kor-QnA \
    --tokenizer_path tokenizers/test-32k \
    --pretrained_model outputs/pretrain-test/final_model \
    --output_dir outputs/sft-test \
    --max_steps 50 \
    --batch_size 2 \
    --logging_steps 10 \
    --save_steps 25

# 실전 학습 (주석 해제)
# python train.py \
#     --mode sft \
#     --dataset BCCard/BCCard-Finance-Kor-QnA \
#     --tokenizer_path tokenizers/ko-128k \
#     --pretrained_model outputs/pretrain-ko/final_model \
#     --output_dir outputs/sft-finance \
#     --num_epochs 3 \
#     --batch_size 4 \
#     --gradient_accumulation_steps 4 \
#     --bf16 \
#     --gradient_checkpointing

In [ ]:
# SFT 학습 로그 확인
import json
from pathlib import Path

log_file = Path("outputs/sft-test/training_log.jsonl")

if log_file.exists():
    print("=" * 60)
    print("SFT 학습 진행 상황")
    print("=" * 60)
    
    with open(log_file) as f:
        logs = [json.loads(line) for line in f]
    
    # 전체 로그 표시
    for log in logs:
        step = log.get('step', 'N/A')
        loss = log.get('loss', 'N/A')
        lr = log.get('learning_rate', 'N/A')
        print(f"Step {step}: Loss={loss:.4f}, LR={lr:.2e}")
else:
    print("⚠️ SFT 학습 로그 파일을 찾을 수 없습니다.")

## 4. 모델 테스트 및 추론

학습된 모델을 로드하고 텍스트를 생성합니다.

In [ ]:
# 모델 및 토크나이저 로드
from moai_llm.modeling.model import MoaiForCausalLM
from transformers import AutoTokenizer
import torch

model_path = "outputs/sft-test/final_model"
tokenizer_path = "tokenizers/test-32k"

print("모델 로딩 중...")
model = MoaiForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

# GPU로 이동
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
model.eval()

print(f"✅ 모델 로드 완료 ({device})")
print(f"파라미터 수: {sum(p.numel() for p in model.parameters()):,}")

In [ ]:
# 텍스트 생성 함수
def generate_text(prompt, max_new_tokens=100, temperature=0.7, top_p=0.9):
    """간단한 텍스트 생성 함수"""
    # 프롬프트 토큰화
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    
    # 생성
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=top_p,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    
    # 디코딩
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

print("✅ 생성 함수 준비 완료")

In [ ]:
# 테스트 1: 간단한 텍스트 생성
print("=" * 60)
print("테스트 1: 간단한 생성")
print("=" * 60)

prompt = "안녕하세요."
result = generate_text(prompt, max_new_tokens=50)

print(f"프롬프트: {prompt}")
print(f"생성 결과:\n{result}")
print()

In [ ]:
# 테스트 2: Q&A 형식 (SFT 학습 형식)
print("=" * 60)
print("테스트 2: Q&A 형식")
print("=" * 60)

# BCCard 학습 형식과 동일하게
question = "신용카드 발급 조건은 무엇인가요?"
prompt = f"<|im_start|>user\n{question}<|im_end|>\n<|im_start|>assistant\n"

result = generate_text(prompt, max_new_tokens=100, temperature=0.7)

print(f"질문: {question}")
print(f"\n전체 생성 결과:\n{result}")
print()

# 답변 부분만 추출
if "<|im_start|>assistant" in result:
    answer = result.split("<|im_start|>assistant")[-1]
    answer = answer.split("<|im_end|>")[0].strip()
    print(f"답변 부분:\n{answer}")

In [ ]:
# 테스트 3: 여러 질문 테스트
print("=" * 60)
print("테스트 3: 다양한 질문")
print("=" * 60)

test_questions = [
    "금융이란 무엇인가요?",
    "대출을 받으려면 어떻게 해야 하나요?",
    "이자율은 무엇인가요?",
]

for i, question in enumerate(test_questions, 1):
    prompt = f"<|im_start|>user\n{question}<|im_end|>\n<|im_start|>assistant\n"
    result = generate_text(prompt, max_new_tokens=80, temperature=0.7)
    
    # 답변 추출
    if "<|im_start|>assistant" in result:
        answer = result.split("<|im_start|>assistant")[-1]
        answer = answer.split("<|im_end|>")[0].strip()
    else:
        answer = result
    
    print(f"\n[질문 {i}] {question}")
    print(f"[답변] {answer}")
    print("-" * 60)

## 5. 모델 평가

간단한 perplexity 계산 및 성능 측정

In [ ]:
# Perplexity 계산
import math

def calculate_perplexity(text):
    """텍스트의 perplexity 계산"""
    inputs = tokenizer(text, return_tensors="pt").to(device)
    
    with torch.no_grad():
        outputs = model(**inputs, labels=inputs["input_ids"])
        loss = outputs.loss.item()
    
    perplexity = math.exp(loss)
    return perplexity, loss

# 테스트 텍스트
test_texts = [
    "안녕하세요. 저는 MOAI-LLM입니다.",
    "금융 서비스는 고객의 자산을 관리하고 증식시키는 것을 목표로 합니다.",
    "asdkfjlasjdf lkjasldkfj alskdjf",  # 무작위 텍스트 (높은 perplexity 예상)
]

print("=" * 60)
print("Perplexity 평가")
print("=" * 60)

for text in test_texts:
    ppl, loss = calculate_perplexity(text)
    print(f"\n텍스트: {text}")
    print(f"Loss: {loss:.4f}")
    print(f"Perplexity: {ppl:.2f}")
    print("-" * 60)

## 6. 대화형 채팅 (간단한 버전)

학습된 모델과 간단하게 대화해볼 수 있습니다.

In [ ]:
def chat(question, max_new_tokens=100, temperature=0.7):
    """간단한 채팅 함수"""
    prompt = f"<|im_start|>user\n{question}<|im_end|>\n<|im_start|>assistant\n"
    result = generate_text(prompt, max_new_tokens=max_new_tokens, temperature=temperature)
    
    # 답변 추출
    if "<|im_start|>assistant" in result:
        answer = result.split("<|im_start|>assistant")[-1]
        answer = answer.split("<|im_end|>")[0].strip()
    else:
        answer = result
    
    return answer

# 테스트
print("=" * 60)
print("채팅 테스트 (입력한 질문에 답변합니다)")
print("=" * 60)

# 여기에 직접 질문을 입력하세요
my_question = "안녕하세요. 자기소개를 해주세요."

answer = chat(my_question, max_new_tokens=100, temperature=0.7)
print(f"\n질문: {my_question}")
print(f"답변: {answer}")

In [ ]:
# 추가 질문 (여기를 수정해서 실행하세요)
my_questions = [
    "머신러닝이 뭔가요?",
    "파이썬으로 Hello World를 출력하려면?",
    "오늘 날씨는 어때요?",
]

for q in my_questions:
    answer = chat(q, max_new_tokens=80)
    print(f"\nQ: {q}")
    print(f"A: {answer}")
    print("-" * 60)

## 7. 모델 저장 및 공유

학습된 모델을 HuggingFace Hub에 업로드할 수 있습니다.

In [ ]:
# HuggingFace Hub 로그인 (선택적)
# from huggingface_hub import login
# login()

# 모델 업로드
# model.push_to_hub("your-username/moai-llm-finance-ko")
# tokenizer.push_to_hub("your-username/moai-llm-finance-ko")

print("💡 HuggingFace Hub에 업로드하려면 위 코드의 주석을 해제하세요.")

## 요약

이 노트북에서 완료한 작업:

✅ **1단계**: 환경 확인 (GPU, 라이브러리)  
✅ **2단계**: 토크나이저 학습 (Wikipedia 데이터셋)  
✅ **3단계**: 사전학습 (Pretrain) - 언어 모델 기초  
✅ **4단계**: 파인튜닝 (SFT) - Q&A 학습  
✅ **5단계**: 추론 테스트 - 텍스트 생성  
✅ **6단계**: 모델 평가 - Perplexity 측정  
✅ **7단계**: 대화형 채팅 테스트  

---

## 다음 단계

### 더 나은 성능을 위해:

1. **토크나이저**: `vocab_size=128000`, 전체 데이터 사용
2. **사전학습**: 더 많은 steps, 더 큰 데이터셋 (Wikipedia 전체, C4 등)
3. **SFT**: 더 많은 epochs, 더 다양한 Q&A 데이터셋
4. **평가**: 벤치마크 데이터셋으로 정량 평가

### 고급 기능:

- **긴 컨텍스트**: YaRN으로 32K → 128K 확장
- **멀티 GPU**: DeepSpeed, FSDP 사용
- **최적화**: Flash Attention, Gradient Checkpointing
- **배포**: vLLM, TensorRT-LLM으로 추론 최적화

### 참고 문서:

- `QUICKSTART.md` - 빠른 시작 가이드
- `USER_GUIDE.md` - 완전한 학습 가이드
- `DATASETS.md` - 데이터셋 선택 가이드
- `ARCHITECTURE.md` - Qwen3 아키텍처 상세

---

**🎉 축하합니다! MOAI-LLM 학습 파이프라인을 완료했습니다!**